In [ ]:
!pip install -U "transformers>=4.42.0" "peft>=0.11.1" "trl>=0.9.4" "accelerate>=0.34.0" bitsandbytes datasets evaluate rouge_score einops scipy


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 12.0 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=bfe0ab89596cd7172b691101da4a3c2e328360f6310bd02c0602c5c547b86a88
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall:

In [ ]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import torch
import time
import tqdm
from transformers import (BitsAndBytesConfig,AutoTokenizer,AutoModelForCausalLM,Trainer)
from huggingface_hub import interpreter_login
from functools import partial
from peft import LoraConfig,get_peft_model, prepare_model_for_kbit_training
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) y


In [ ]:
hugging_face_dataset="neil-code/dialogsum-test"
dataset=load_dataset(hugging_face_dataset)

README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1999 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/499 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/499 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1999
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 499
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 499
    })
})

In [ ]:
dataset['train'][0]

{'id': 'train_0',
 'dialogue': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.",
 'summary': "Mr. Smith'

In [ ]:
compute_dtype=getattr(torch,"float16")
bnb_config=BitsAndBytesConfig(
    Load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype
)

In [ ]:
model_name="microsoft/phi-2"
device_map={"":0}
original_model=AutoModelForCausalLM.from_pretrained(model_name,
                                                    quantization_config=bnb_config,
                                                    device_map=device_map,
                                                    trust_remote_code=True,
                                                    use_auth_token=True)

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(model_name,
                                        trust_remote_code=True,
                                        padding_side="left",
                                        add_eos_token=True,
                                        add_bos_token=True,
                                        use_fast=False)
tokenizer.pad_token=tokenizer.eos_token

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
def gen(model,tokenizer,input,max_len):
  inputs=tokenizer(input,return_tensors="pt")
  inputs={k:v.to("cuda") for k,v in inputs.items()}
  generated_ids=model.generate(**inputs,max_new_tokens=max_len)
  generated_output=tokenizer.batch_decode(generated_ids,skip_special_tokens=True)
  return generated_output


In [ ]:
%%time
from transformers import set_seed
seed=42
set_seed(seed)
index=10
prompt=dataset['test'][index]['dialogue']
summary=dataset['test'][index]['summary']
formated_prompt=f"Instruct:Summarize the following Conversation.\n{prompt}\nOutput:\n"

res=gen(original_model,tokenizer,formated_prompt,200)
output = res[0].split('Output:\n')[1]

dash_line='-'*100

print(f"INPUT PROMPT:\n{formated_prompt}")
print(dash_line)
print(f"SUMMARY:\n{summary}")
print(dash_line)
print(f"MODEL GENERATION - ZERO SHOT:\n{output}")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


INPUT PROMPT:
Instruct:Summarize the following Conversation.
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday
Output:

----------------------------------------------------------------------------------------------------
SUMMARY:
#Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.
-------------------------------------------------------------

In [ ]:
def create_prompt_format(sample):

    INTRO_BLURB = "Below is an instruction..."
    INSTRUCTION_KEY = "### Instruct: Summarize the below conversation"
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"

    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"

    input_context = sample["dialogue"]
    response = f"{RESPONSE_KEY}\n{sample['summary']}"
    end = f"{END_KEY}"

    parts = [blurb, instruction, input_context, response, end]
    formatted_prompt = "\n\n".join(parts)

    sample["text"] = formatted_prompt
    return sample


In [ ]:
def get_max_length(model):
  config=model.config
  max_length=None

  for name in ['n_positions','max_position_embedddings','seq_length']:
    max_length=getattr(config,name,None)
    if max_length:
      print(f"Found max length :{max_length}")
      break
  if not max_length:
    max_length=1024
    print(f"Using default max length {max_length}")
  return max_length


def preprocess_batch(batch,tokenizer,max_length):

  return tokenizer(
      batch['text'],
      max_length=max_length,
      truncation=True
  )


def preprocess_dataset(tokenizer:AutoTokenizer,max_length:int,seed,dataset):

  print(f"Preprocessing Dataset")

  dataset=dataset.map(create_prompt_format)
  preprocessing_fun=partial(preprocess_batch,max_length=max_length,tokenizer=tokenizer)
  dataset=dataset.map(preprocessing_fun,batched=True,remove_columns=['id', 'topic', 'dialogue', 'summary'])
  dataset=dataset.filter(lambda sample :len(sample["input_ids"])<max_length)

  dataset=dataset.shuffle(seed)
  return dataset



In [ ]:
max_length=get_max_length(original_model)

train_dataset=preprocess_dataset(tokenizer,max_length,seed,dataset['train'])
eval_dataset=preprocess_dataset(tokenizer,max_length,seed,dataset['validation'])

Using default max length 1024
Preprocessing Dataset


Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1999 [00:00<?, ? examples/s]

Preprocessing Dataset


Map:   0%|          | 0/499 [00:00<?, ? examples/s]

Map:   0%|          | 0/499 [00:00<?, ? examples/s]

Filter:   0%|          | 0/499 [00:00<?, ? examples/s]

In [ ]:
original_model=prepare_model_for_kbit_training(original_model)

config=LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q_proj",
                    "k_proj",
                    "v_proj",
                    "dense"],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM"

)
original_model.gradient_checkpointing_enable()
peft_model=get_peft_model(original_model,config)

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"Trainable params: {trainable_params}")
    print(f"All params: {all_param}")
    print(f"Trainable%: {100 * trainable_params / all_param:.2f}%")
    return trainable_params, all_param


In [ ]:
print(print_number_of_trainable_model_parameters(peft_model))

Trainable params: 20971520
All params: 1542364160
Trainable%: 1.36%
(20971520, 1542364160)


In [ ]:
import transformers

In [ ]:
from transformers import TrainingArguments
output_dir= f'./peft-dialogue-summary-training-{str(int(time.time()))}'
peft_training_args=TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=1000,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=25,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    eval_strategy="steps",
    eval_steps=25,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
)
peft_model.config.use_cache=False
peft_trainer=Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
peft_trainer.train()

In [ ]:
base_model_id='microsoft/phi-2'
base_model=AutoModelForCausalLM.from_pretrained(base_model_id,
                                                device_map='auto',
                                                quantization_config=bnb_config,
                                                trust_remote_code=True,
                                                use_auth_token=True)

/usr/local/lib/python3.12/dist-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
eval_tokenizer=AutoTokenizer.from_pretrained(base_model_id,add_bos_token=True,trust_remote_code=True,use_fast=False)
eval_tokenizer.pad_token=eval_tokenizer.eos_token

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/Colab Notebooks/my_folder/checkpoint-1000",torch_dtype=torch.float16,is_trainable=False)

In [ ]:
from transformers import set_seed
seed=42

In [ ]:
# set_seed(seed)
# index=5
# dialogue=dataset['test'][index]['dialogue']
# summary=dataset['test'][index]['summary']
# prompt=f'Instruct: Summarize the following Conversation.\n {dialogue}\noutput:\n'

# peft_model_res = gen(ft_model,eval_tokenizer,prompt,100)
# print(peft_model_res)
# peft_model_output = peft_model_res[0].split('output:\n')[1]
# #print(peft_model_output)
# prefix, success, result = peft_model_output.partition('###')

# dash_line = '-'.join('' for x in range(100))
# print(dash_line)
# print(f'INPUT PROMPT:\n{prompt}')
# print(dash_line)
# print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
# print(dash_line)
# print(f'PEFT MODEL:\n{prefix}')





In [ ]:
import pandas as pd
dialogues=dataset['test'][0:10]['dialogue']
human_baseline_summaries=dataset['test'][0:10]['summary']

original_model_summaries=[]
peft_model_summaries=[]

for idx,dialogue in enumerate(dialogues):
  human_baseline_text_output=human_baseline_summaries[idx]
  prompt=f"Instruct:Summarize the following conversation.\n{dialogue}\nOutput:\n"

  original_model_res=gen(base_model,eval_tokenizer,prompt,200)
  original_model_text_output=original_model_res[0].split('Output:\n')[1]

  peft_model_res=gen(ft_model,eval_tokenizer,prompt,200)
  peft_model_output=peft_model_res[0].split('output:\n')[1]
  print(peft_model_output)
  peft_model_text_output,success,result=peft_model_output.partition("###")

  original_model_summaries.append(original_model_text_output)
  peft_model_summaries.append(peft_model_text_output)


zipped_summaries=list(zip(human_baseline_summaries,original_model_summaries,peft_model_summaries))

df=pd.DataFrame(zipped_summaries,columns=['human_baseline_summaries', 'original_model_summaries', 'peft_model_summaries'])



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import evaluate

rouge=evaluate.load('rouge')
original_model_results=rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)
peft_model_results=rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)
print('ORIGINAL MODEL:')
print(original_model_results)
print('PEFT MODEL:')
print(peft_model_results)

print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement=(np.array(list(peft_model_results.values()))-np.array(list(original_model_results.values())))
for key,value in zip(peft_model_results.keys(),improvement):
  print(f'{key}: {value*100:2f}%')